**NHÓM C2**

# Code phụ: Selenium trên ứng dụng Firefly III

Mục đích: Minh hoạ việc tạo Asset Account và kiểm tra kết quả tạo tự động bằng Selenium với demo riêng biệt trước khi tích hợp vào Locust

**Môi trường thực thi:**

- MacOS Sonoma 14.0
- CPU Apple M1
- Python 3.10.10
- Trình duyệt Microsoft Edge Version 118.0.2088.76 (Official build) (arm64)

## Cài đặt Selenium

In [1]:
!pip install selenium

## Import thư viện

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import datetime
import os

## Server IP & testing URL

In [3]:
# IP của server được deploy trên Google Cloud 
# (tài khoản deploy có thể hết ngân sách sau deadline nên có thể làm web không chạy được)
server_ip = "34.16.184.203"

# URL đến trang đăng nhập
login_url = f"http://{server_ip}/login"

# URL đến trang danh sách tài khoản dạng asset
asset_account_list_url = f"http://{server_ip}/accounts/asset"

# URL đến trang tạo tài khoản dạng asset
create_asset_account_url = f"http://{server_ip}/accounts/create/asset"

print(login_url)
print(asset_account_list_url)
print(create_asset_account_url)

http://34.16.184.203/login
http://34.16.184.203/accounts/asset
http://34.16.184.203/accounts/create/asset


## Hàm đổi màu phần tử trên trang web

In [4]:
def change_element_color(browser, element):
    browser.execute_script(
        "arguments[0].style.backgroundColor = 'lightgreen';", 
        element
    )

## Hàm đọc file văn bản

In [5]:
def read_file(file_path):
    try:
        with open(file_path, 'r') as file:
            file_contents = file.read()
        return file_contents
    except FileNotFoundError:
        return f"File not found: {file_path}"
    except Exception as e:
        return f"An error occurred: {str(e)}"

## Tài khoản đăng nhập vào hệ thống

In [6]:
# Tạo file lưu password (để không trình chiếu password khi demo)
# !touch "password.pass" # Lệnh này không gây ảnh hưởng nếu file đã tồn tại

email = "20120406@student.hcmus.edu.vn"
password = read_file("password.pass")

## Microsoft Edge Remote Debugging Port (tránh bị nhận diện phần mềm automation)

In [7]:
browser_instance_data_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKI/Kiem thu phan mem/Project/Final-Project/Performance Testing/Source/browser_instance_data"
port_to_run_browser = 40000

command = f'''open -na "Microsoft Edge.app" --args
            --remote-debugging-port={port_to_run_browser}
            --user-data-dir="{browser_instance_data_dir}"'''

os.system(command.replace("\n", " "))

0

## Kết nối Selenium đến trình duyệt Edge ở port được mở

In [8]:
options = webdriver.EdgeOptions()
options.add_experimental_option("debuggerAddress", f"localhost:{port_to_run_browser}")
# options.add_argument("headless") # Chạy Selenium ở chế độ headless (không hiển thị giao diện browser)

browser = webdriver.Edge(options=options)
browser.maximize_window() # Phóng to cửa sổ trình duyệt

## Đi đến trang đăng nhập

In [9]:
def go_to_login_page(browser, login_url):
    browser.get(login_url)

go_to_login_page(browser, login_url)

## Đăng nhập

In [10]:
def enter_email_and_password(browser, email, password):
    username_input_field = browser.find_element(
        By.CSS_SELECTOR, 'input[name="email"]'
    )
    change_element_color(browser, username_input_field)
    username_input_field.send_keys(email)
    
    password_input_field = browser.find_element(
        By.CSS_SELECTOR, 'input[name="password"]'
    )
    change_element_color(browser, password_input_field)
    password_input_field.send_keys(password)

    sign_in_button = browser.find_element(
        By.CSS_SELECTOR, 
        'button[type="submit"]'
    )
    change_element_color(browser, sign_in_button)
    sign_in_button.click()

enter_email_and_password(browser, email, password)

## Đi đến trang tạo Asset Account

In [11]:
# Đi đến trang tạo Asset Account
def go_to_create_asset_account_page(browser):
    browser.get(create_asset_account_url)

go_to_create_asset_account_page(browser)

## Hàm tạo tên Asset Account

In [12]:
def create_asset_account_name():
    # Lấy ngày giờ hiện tại
    current_datetime = datetime.datetime.now()

    # Format ngày giờ theo dạng "hh:mm:ss dd/mm/yyyy"
    formatted_datetime = current_datetime.strftime("%H:%M:%S %d/%m/%Y")

    email = f'Asset Account {formatted_datetime}'

    return email

create_asset_account_name_example = create_asset_account_name()
print(f"Example: {create_asset_account_name_example}")

Example: Asset Account 21:38:24 09/01/2024


## Nhập thông tin Asset Account

In [13]:
def enter_asset_account_name(browser, asset_account_name):
    name_field = browser.find_element(
        By.CSS_SELECTOR, 'input[placeholder="Name"]'
    )
    change_element_color(browser, name_field)
    name_field.send_keys(asset_account_name)

asset_account_name = create_asset_account_name()
print(f"Asset account name: {asset_account_name}")

enter_asset_account_name(browser, asset_account_name)

Asset account name: Asset Account 21:38:28 09/01/2024


## Nhấn Store new asset account

In [14]:
def store_new_asset_account(browser):
    new_asset_account_button = browser.find_element(
        By.XPATH,
        '//button[contains(text(), "Store new asset account")]'
    )
    
    change_element_color(browser, new_asset_account_button)
    new_asset_account_button.click()

store_new_asset_account(browser)

## Kiểm tra tạo thành công

In [15]:
def was_asset_account_successfully_created(browser, asset_account_name):
    # Đến trang danh sách asset account
    browser.get(asset_account_list_url)

    # Thử tìm trên trang danh sách asset account xem có asset account vừa tạo không
    # Nếu có thì trả về True, không thì trả về False
    try:
        generated_account = browser.find_element(
            By.XPATH,
            f'//a[contains(text(), "{asset_account_name}")]'
        )
        
        change_element_color(browser, generated_account)
        
        print(f"Test case: {asset_account_name} - PASSED")
        return True
    except:
        print(f"Test case: {asset_account_name} - FAILED")
        return False
    
print(was_asset_account_successfully_created(browser, asset_account_name))

Test case: Asset Account 21:38:28 09/01/2024 - PASSED
True
